In [35]:
from ezc3d import c3d
import numpy as np
import collections, copy
from sklearn.preprocessing import MinMaxScaler


In [36]:
# file = r"C:\Users\anna\Documents\gitHub\NTU\InterpolationModel\data\c3d\SN268\SN268_0023_10m_01.c3d"
file = r"C:\Users\Administrator\Documents\gitHub\NTU\InterpolationModel\data\SN268\SN268_0023_10m_01.c3d"




In [37]:
class standardise:
    def __init__(self): 
        self.mapping_anatomical_group_to_marker = {
            'head':[
            'RHEAD',
            'RTEMP',
            'LHEAD',
            'LTEMP'],

            'shoulder':[
            'RACR',
            'LACR'],

            'torso':[
            'STER',
            'XPRO',
            'C7',
            'T4',
            'T8',
            'T10'],
            

            'pelvis':[
            'LPSIS',
            'RPSIS',
            'RASIS',
            'LASIS',
            'LICR',
            'RICR'],

            'arms':[    
            'RHME',
            'RHLE',
            'RUA1',
            'RUA2',
            'RUA3',
            'RUA4',
            'LHME',
            'LHLE',
            'LUA1',
            'LUA2',
            'LUA3',
            'LUA4'],

            'wrist':[
            'LFA1',
            'LFA2',
            'LFA3',
            'LRSP',
            'LUSP',
            'RRSP',
            'RUSP',
            'RFA3',
            'RFA2',
            'RFA1'],

            'hand':[
            'LCAP',
            'LHMC1',
            'LHMC2',
            'LHMC3',
            'RCAP',
            'RHMC1',
            'RHMC2',
            'RHMC3'],

            'fingers':[
            'LTHUMB',
            'RTHUMB',
            'RMIDDLE',
            'LMIDDLE'],

            'thigh':[
            'LFLE',
            'LFME',
            'LTH1',
            'LTH2',
            'LTH3',
            'LTH4',
            'RFLE',
            'RFME',
            'RTH1',
            'RTH2',
            'RTH3',
            'RTH4'],

            'calves':[
            'LSK1',
            'LSK2',
            'LSK3',
            'LSK4',
            'LFAL',
            'LTAM',
            'RSK1',
            'RSK2',
            'RSK3',
            'RSK4',
            'RFAL',
            'RTAM'],

            'foot':[
            'RFCC',
            'RFMT5',
            'RFMT2',
            'RFMT1',
            'LFCC',
            'LFMT5',
            'LFMT2',
            'LFMT1']
            }
        # all tasks
        self.all_tasks = [ 
            "key_stand",
            "back", 
            "mouth", 
            "head", 
            "grasp", 
            "lateral", 
            "towel", 
            "step_down", 
            "step_up", 
            "kerb", 
            "balance", 
            "tug", 
            "10m", 
            "static", 
            "balance_static"]

        # different task needs different markerset; map accordingly by colour
        self.mapping_task_to_color = {
            'towel':'blue', 
            'grasp':'green', 
            'lateral':'green', 
            'mouth':'green', 
            'head':'green', 
            'back':'green', 
            'tug':'pink', 
            'key_stand':'pink', 
            'balance':'yellow', 
            'kerb':'yellow', 
            'step':'yellow', 
            '10m':'yellow',
            'step_down':'yellow',
            'step_up':'yellow',
            'balance_static':'yellow'}
        
        # Experimentation 1: travelling sales man (top down)
        self.mapping_color_to_anatomical_group = {
            'blue':['head', 'shoulder', 'torso', 'pelvis', 'arms', 'wrist', 'hand'],
            'green':['head', 'shoulder', 'torso', 'pelvis', 'arms', 'wrist', 'hand', 'fingers'],
            'pink':['head', 'shoulder', 'torso', 'pelvis', 'arms', 'wrist', 'hand', 'thigh', 'calves', 'foot'],
            'yellow':['head', 'shoulder', 'torso', 'pelvis', 'arms', 'wrist', 'thigh', 'calves', 'foot']
            }
        

    def transform_data(self, file, task):
        '''load file and rearrange the markers that are highly correlated together'''
        # load c3d file
        c = c3d(file)

        # get all markers name in the file and their coordinates
        name_of_markers = c["parameters"]["POINT"]["DESCRIPTIONS"]["value"]
        coordinates_of_markers = c["data"]["points"][0:3,:,:]

        # map task to color to anatomical group to marker in the anatomical group
        anatomical_group = self.mapping_color_to_anatomical_group[self.mapping_task_to_color[task]]
        transformed_data =  None
        for group in anatomical_group:
            for marker in self.mapping_anatomical_group_to_marker[group]:
                idx =  name_of_markers.index(marker)

                # start forming the sequence according to self.mapping_color_to_anatomical_group
                if transformed_data is None:
                    transformed_data = coordinates_of_markers[:,idx:idx+1,:]
                else:
                    transformed_data = np.concatenate((transformed_data, coordinates_of_markers[:,idx:idx+1,:]), axis = 1)

        return transformed_data

        

In [67]:
def minmax_znorm(data, znorm = True, minmax = True):
    '''minmax after z-score normalisation (standardisation) of the data'''
    manipulate_data = copy.copy(data)
    axes = ['x', 'y', 'z']

    # for inversion of znorm later
    znorm_scaling = collections.defaultdict()
    # for inversion of minmax later
    minmax_scaling = collections.defaultdict()

    for i, axis in enumerate(axes):
        data_in_each_axis = manipulate_data[i:i+1,:,:]
        if znorm == True:
            # record down the standardisation factors for z-score normalisation 
            znorm_scaling["avg_" + axis] = np.mean(data_in_each_axis)
            znorm_scaling["sd_" + axis] = np.std(data_in_each_axis)
            # z-score normalise the data
            manipulate_data[i:i+1,:,:] = ((data_in_each_axis - znorm_scaling["avg_" + axis]) / znorm_scaling["sd_" + axis])
        

        if minmax == True:
            # record down the scaling factors for minmax
            minmax_scaling["min_" + axis] = np.min(data_in_each_axis)
            minmax_scaling["max_" + axis] = np.max(data_in_each_axis)
            # minmax the normalised data
            manipulate_data[i:i+1,:,:] = ((data_in_each_axis - minmax_scaling["min_" + axis]) / (minmax_scaling["max_" + axis] - minmax_scaling["min_" + axis]))

    return znorm_scaling, minmax_scaling, manipulate_data
    # code for minmax scaler if wanted for Experimentation 2: Per-Axis Scaling - preserves the individual characteristics of each time series within an axis.
        # znorm_data_in_each_axis = znorm_data[i,:,:]
        # scaler = MinMaxScaler(feature_range=(0, 1))
        # minmax_znorm_data = scaler.fit_transform(znorm_data_in_each_axis)
        #     minmax_znorm_data = np.dstack((minmax_znorm_data, scaler.fit_transform(znorm_data_in_each_axis)))

In [69]:
standardise_data = standardise()
data = standardise_data.transform_data(file, '10m')
znorm_scaling, minmax_scaling, scaled_data = minmax_znorm(data, znorm = True, minmax = True)


defaultdict(None, {'avg_x': 560.8887880817874, 'sd_x': 1257.702716550006})
-5.132630741634556e-17
defaultdict(None, {'avg_x': 560.8887880817874, 'sd_x': 1257.702716550006, 'avg_y': 212.78185082997152, 'sd_y': 150.01928343389503})
1.0451902601146732e-16
defaultdict(None, {'avg_x': 560.8887880817874, 'sd_x': 1257.702716550006, 'avg_y': 212.78185082997152, 'sd_y': 150.01928343389503, 'avg_z': 723.1271584790336, 'sd_z': 423.8417062096214})
8.958773658125771e-17


In [58]:
scaled_data

array([[[6.37156914e-02, 6.57766115e-02, 6.76638894e-02, ...,
         9.78071643e-01, 9.80227844e-01, 9.82267340e-01],
        [5.75934048e-02, 5.96056893e-02, 6.15815693e-02, ...,
         9.71808321e-01, 9.73915598e-01, 9.76033718e-01],
        [6.40593614e-02, 6.55179292e-02, 6.74946210e-02, ...,
         9.78212914e-01, 9.80343291e-01, 9.82289287e-01],
        ...,
        [5.18832498e-02, 5.85901578e-02, 6.53135919e-02, ...,
         9.60635458e-01, 9.60614610e-01, 9.60609006e-01],
        [6.17087497e-02, 6.83433199e-02, 7.49593735e-02, ...,
         9.68144341e-01, 9.68178598e-01, 9.68249260e-01],
        [6.26122126e-02, 6.95931766e-02, 7.63317226e-02, ...,
         9.67040574e-01, 9.67057022e-01, 9.67056760e-01]],

       [[4.90351610e-01, 4.89858556e-01, 4.89734233e-01, ...,
         4.42909439e-01, 4.42650345e-01, 4.42661348e-01],
        [4.34382567e-01, 4.34027317e-01, 4.33522009e-01, ...,
         3.88535045e-01, 3.88390933e-01, 3.88146813e-01],
        [5.88397082e-01, 

AxisError: axis 1 is out of bounds for array of dimension 1

In [73]:
znorm_scaling

defaultdict(None,
            {'avg_x': 560.8887880817874,
             'sd_x': 1257.702716550006,
             'avg_y': 212.78185082997152,
             'sd_y': 150.01928343389503,
             'avg_z': 723.1271584790336,
             'sd_z': 423.8417062096214})

In [74]:
minmax_scaling

defaultdict(None,
            {'min_x': -1.8644621262192393,
             'max_x': 1.841410240004742,
             'min_y': -2.09182055854054,
             'max_y': 2.133001813100659,
             'min_z': -1.694439844491011,
             'max_z': 1.8353943308694023})

In [75]:
scaled_data

array([[[-1487.07873535, -1477.47302246, -1468.67663574, ...,
          2774.62988281,  2784.6796875 ,  2794.18554688],
        [-1515.61401367, -1506.23498535, -1497.02563477, ...,
          2745.43725586,  2755.2590332 ,  2765.13134766],
        [-1485.47692871, -1478.67871094, -1469.46557617, ...,
          2775.28833008,  2785.21777344,  2794.2878418 ],
        ...,
        [-1542.22839355, -1510.96826172, -1479.63110352, ...,
          2693.36181641,  2693.26464844,  2693.23852539],
        [-1496.43286133, -1465.5098877 , -1434.67321777, ...,
          2728.35986328,  2728.51953125,  2728.84887695],
        [-1492.22192383, -1459.68444824, -1428.27685547, ...,
          2723.21533203,  2723.29199219,  2723.29077148]],

       [[  209.75564575,   209.44314575,   209.36434937, ...,
           179.68656921,   179.52235413,   179.52932739],
        [  174.28219604,   174.05703735,   173.73677063, ...,
           145.22381592,   145.13247681,   144.97775269],
        [  271.89733887, 

In [ ]:
task = '10m'

In [29]:
def reorganize_data(task, coordinates_of_markers):
    

                
    ]

SyntaxError: expected ':' (3775832973.py, line 1)

In [27]:
# markers_grouping = markers_dict = {
#     'R_head':[
#         'RHEAD',
#         'RTEMP'],

#     'L_head':[
#         'LHEAD',
#         'LTEMP'],

#     'R_shoulder':[
#         'RACR'],

#     'L_shoulder':[
#         'LACR'],

#     'front_torso':[
#         'STER',
#         'XPRO'],

#     'back_torso':[    
#         'C7',
#         'T4',
#         'T8',
#         'T10'],
    

#     'R_pelvis':[
#         'RPSIS',
#         'RICR',
#         'RASIS'],

#     'L_pelvis':[
#         'LPSIS',
#         'LASIS',
#         'LICR'],

#     'R_arm':[    
#         'RHME',
#         'RHLE',
#         'RUA1',
#         'RUA2',
#         'RUA3',
#         'RUA4'],
    
#     'L_arm':[
#         'LHME',
#         'LHLE',
#         'LUA1',
#         'LUA2',
#         'LUA3',
#         'LUA4'],

#     'R_wrist':[
#         'RRSP',
#         'RUSP',
#         'RFA3',
#         'RFA2',
#         'RFA1'],

#     'L_wrist':[
#         'LFA1',
#         'LFA2',
#         'LFA3',
#         'LRSP',
#         'LUSP'],

#     'R_hand':[
#         'RCAP',
#         'RHMC1',
#         'RHMC2',
#         'RHMC3'],
    
#     'L_hand':[
#         'LCAP',
#         'LHMC1',
#         'LHMC2',
#         'LHMC3'],

#     'R_fingers':[
#         'RTHUMB',
#         'RMIDDLE'],
    
#     'L_fingers':[
#         'LTHUMB',
#         'LMIDDLE'],

#     'R_thigh':[
#         'RFLE',
#         'RFME',
#         'RTH1',
#         'RTH2',
#         'RTH3',
#         'RTH4'],
    
#     'L_thigh':[
#         'LFLE',
#         'LFME',
#         'LTH1',
#         'LTH2',
#         'LTH3',
#         'LTH4'],

#     'R_calf':[
#         'RSK1',
#         'RSK2',
#         'RSK3',
#         'RSK4',
#         'RFAL',
#         'RTAM'],
        
#     'L_calf':[
#         'LSK1',
#         'LSK2',
#         'LSK3',
#         'LSK4',
#         'LFAL',
#         'LTAM'],

#     'R_foot':[
#         'RFCC',
#         'RFMT5',
#         'RFMT2',
#         'RFMT1'],

#     'L_foot':[
#         'LFCC',
#         'LFMT5',
#         'LFMT2',
#         'LFMT1']
#     }
